In [1]:
pip install selenium==4.10.0

  Using cached selenium-4.10.0-py3-none-any.whl (6.7 MB)
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.0
    Uninstalling selenium-4.8.0:
      Successfully uninstalled selenium-4.8.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time 
import datetime
import pandas as pd

url = "https://twitter.com"

options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

driver.get(url)

driver.implicitly_wait(3) 

# 로그인
login_button = driver.find_element(By.XPATH, "//span[contains(text(), '로그인')]")
login_button.click()

#아이디 비번 입력
with open(r"C:\Users\Gyoungwon Jeong\Desktop\sol_project_4\10. data crawling\twitter password.txt", "r") as twitter_login:
    lines = twitter_login.readlines()
    id = lines[0].strip()
    pw = lines[1].strip()
    
input_id = driver.find_element(By.XPATH, "//input[@name='text']") 
input_id.send_keys(id)
input_id.send_keys(Keys.ENTER)

input_pw = driver.find_element(By.XPATH, "//input[@name='password']")
input_pw.send_keys(pw)
input_pw.send_keys(Keys.ENTER)

# 최신 트렌드 가져오기
trends_for_you = driver.find_element(By.XPATH, "//div[@aria-label='Timeline: Trending now']")
trend_for_you = trends_for_you.find_elements(By.XPATH, ".//span[contains(@class, 'css-901oao') and not(contains(text(), 'Trends')) and not(contains(text(), 'posts')) and not(contains(text(), 'Trending')) and not(contains(text(), 'Show'))]")

trends = []

for trend in trend_for_you:
    trends.append(trend.text)

print(trends)

#검색 쿼리 가져오기
search_queries = []

today_date = datetime.datetime.now()
yesterday_date = today_date - datetime.timedelta(days=1)

start_date = yesterday_date.strftime("%Y-%m-%d")
end_date = today_date.strftime("%Y-%m-%d") 

for trend in trends:
    search_query = trend + ' lang:ko ' + 'until:' + end_date + ' since:' + start_date
    search_queries.append(search_query)

print(search_queries)

#검색어에 트렌드 검색
input_search = driver.find_element(By.XPATH, "//input[@aria-label='Search query']")
input_search.send_keys(search_queries[0])
input_search.send_keys(Keys.ENTER) 

seen_tweets = set()
tweet = []

while True:
    current_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #현재 스크롤 높이
    tweet_elements = driver.find_elements(By.XPATH, "//div[@class='css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0' and @data-testid='tweetText']")
    
    for tweet_element in tweet_elements:
        tweet_content = tweet_element.text.strip()
        if tweet_content not in seen_tweets:
            tweet.append(tweet_content) #현재 페이지 트윗 
            seen_tweets.add(tweet_content) #축적 트윗
    
    #print("Current Page Tweets Count:", len(tweet))
    #print("Seen Tweets Count:", len(seen_tweets))
    
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") #스크롤 내리기
    time.sleep(3)  # 콘텐츠 로딩 기다리기
    new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤 내린 후 높이
    if new_scroll_height == current_scroll_height:
        break

#트렌드 계속 하나씩 수집하기
for search_query in search_queries[1:]:
    input_search = driver.find_element(By.XPATH, "//input[@data-testid='SearchBox_Search_Input']")
    # 검색 창 내용을 선택하고 지우기
    input_search.send_keys(Keys.CONTROL + "a")
    input_search.send_keys(Keys.DELETE)
    input_search.send_keys(search_query)
    input_search.send_keys(Keys.ENTER)
    
    while True:
        current_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #현재 스크롤 높이
        tweet_elements = driver.find_elements(By.XPATH, "//div[@class='css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0' and @data-testid='tweetText']")
        
        for tweet_element in tweet_elements:
            tweet_content = tweet_element.text.strip()
            if tweet_content not in seen_tweets:
                tweet.append(tweet_content) #현재 페이지 트윗 
                seen_tweets.add(tweet_content) #축적 트윗
        
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") #스크롤 내리기
        time.sleep(3)  # 콘텐츠 로딩 기다리기
        new_scroll_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤 내린 후 높이
        if new_scroll_height == current_scroll_height:
            break
    
# 데이터프레임에 넣기
twitter_df = pd.DataFrame({'content':tweet})
print(twitter_df)

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


['아무런 잘못', '#오펜하이머', '오펜하이머 - 크리스토퍼 놀란 감독 작품', 'Promoted by 유니버설 픽쳐스', 'Promoted by 유니버설 픽쳐스', '민규 면도컷', '지드래곤', '사건 당사자', '유노윤호', '당신의 19금 성향', '마스크걸', '요즘 20', '김규빈 기자님']
['아무런 잘못 lang:ko until:2023-08-18 since:2023-08-17', '#오펜하이머 lang:ko until:2023-08-18 since:2023-08-17', '오펜하이머 - 크리스토퍼 놀란 감독 작품 lang:ko until:2023-08-18 since:2023-08-17', 'Promoted by 유니버설 픽쳐스 lang:ko until:2023-08-18 since:2023-08-17', 'Promoted by 유니버설 픽쳐스 lang:ko until:2023-08-18 since:2023-08-17', '민규 면도컷 lang:ko until:2023-08-18 since:2023-08-17', '지드래곤 lang:ko until:2023-08-18 since:2023-08-17', '사건 당사자 lang:ko until:2023-08-18 since:2023-08-17', '유노윤호 lang:ko until:2023-08-18 since:2023-08-17', '당신의 19금 성향 lang:ko until:2023-08-18 since:2023-08-17', '마스크걸 lang:ko until:2023-08-18 since:2023-08-17', '요즘 20 lang:ko until:2023-08-18 since:2023-08-17', '김규빈 기자님 lang:ko until:2023-08-18 since:2023-08-17']
                                               content
0    안녕하십니까.\n사건 당사자입니다. 진지하게 사죄드립니다...\n지난 8월 14일 ..

In [7]:
twitter_df.to_csv('확인용3')

In [10]:
'C:\Users\Gyoungwon Jeong\Desktop\sol_project_4\10. data crawling'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3144549665.py, line 1)

In [8]:
cd

C:\Users\Gyoungwon Jeong


In [11]:
twitter_df.to_csv('확인용5.csv')

In [15]:
df1 = pd.read_csv('C:/Users/Gyoungwon Jeong/Desktop/sol_project_4/10. data crawling/확인용3.csv')

In [16]:
df2 = pd.read_csv('C:/Users/Gyoungwon Jeong/Desktop/sol_project_4/10. data crawling/확인용2')

In [17]:
df3 = pd.read_csv('C:/Users/Gyoungwon Jeong/Desktop/sol_project_4/10. data crawling/확인용4.csv')

In [18]:
df4 = pd.read_csv('C:/Users/Gyoungwon Jeong/Desktop/sol_project_4/10. data crawling/확인용5.csv')

In [37]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)


In [41]:
df = df['content']

In [42]:
df

0                                         세븐틴 포카 전멤버 양도해요
1       세븐틴 svt 도겸 포카 양도 sell 일괄 우대\n\n오드투유 캐럿랜드 트레카 멤...
2       승관이 위버스에 댓글 단 멤버들\n보고싶다고 해주면서도\n슬픔이나 아쉬웠다는것 말고...
3       솔직히 내가 세븐틴 몇년 좋아해오면서\n제일 믿음가는 멤버 부승관이였음\n그냥 이유...
4       나 진짜 방금 승관이 글 보는 것도 멤버들 댓글 보는 것도 따뜻하다 정도만 인식했는...
                              ...                        
1674    당신의 19금 성향은?\n──────────────────\nmedicrider님은...
1675    당신의 19금 성향은?\n──────────────────\n초기설정님은 정상인입니...
1676    당신의 19금 성향은?\n──────────────────\nㅇㅇㅇㅇ님은 재갈입니다...
1677    당신의 19금 성향은?\n──────────────────\n카미오카 유리님은 [검...
1678    당신의 19금 성향은?\n──────────────────\n애님은 돔입니다.\nh...
Name: content, Length: 1679, dtype: object

In [43]:
df.to_csv('sample_tweets.csv')